In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_C_057, input_features_C_057 = pre_models.Input_C_057('weights/')
    
    outputs = ['Input_C_057']

    preprocessings = ['min_max_normalization']

    models = [model_C_057]

    input_features = [input_features_C_057]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=7
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_057.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_055', 'Input_C_135', 'Input_C_052', 'Input_C_054',
       'Input_C_056', 'Input_C_059', 'Input_C_053'], dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.004828187065605178
[0.005541502476193872, 0.004351071938119547, 0.005371355274324799, 0.0052931168491350015, 0.0042511358609742065, 0.004562474962971541, 0.005097942475172927, 0.004794913251169597, 0.004946849397535215, 0.004071508170455074]


test_r2_ave: 0.5741698281643208
[0.3735267610086245, 0.6193109854791077, 0.4790528614262891, 0.5247398835295266, 0.6536907032087198, 0.6446636548364408, 0.5166182079606012, 0.6138813746126164, 0.608341226223212, 0.7078726233580708]


## Linear Regression Baseline

In [9]:
lr_rmse_ave_fea(skew_data,7)

train_rmse: 0.004042087916713229
test_rmse: 0.0042089979115283935
test_r2: 0.6829602163837639


## Lasso

In [10]:
laso_rmse_ave(skew_data[title],y,9e-05)

test_rmse_ave: 0.0041904108831711685
[0.004287509088716964, 0.003776694510842413, 0.00480016563914731, 0.004305953432606997, 0.0038289321858901337, 0.0041579762023212785, 0.0044878348324749115, 0.004106011221814918, 0.004378551486645267, 0.003774480231251494]


test_r2_ave: 0.6815897086733098
[0.6249772897576982, 0.7131852174572721, 0.5839568773674868, 0.685480870668556, 0.7190626184048907, 0.7048772824351657, 0.6253944941082511, 0.7168611221161302, 0.6931604028396188, 0.7489409115780302]


## ElasticNet

In [11]:
ElasticNet_rmse_ave(skew_data[title],y,[0.006],0.01)

test_rmse_ave: 0.004193702972113819
[0.004286462440973182, 0.003775651289664121, 0.004802520122243878, 0.0043315897036147275, 0.0038299855778753163, 0.004159692826994745, 0.004486420957920593, 0.0041066982476830015, 0.004378165143431671, 0.00377984341073695]


test_r2_ave: 0.6811157193753848
[0.6251603651778102, 0.7133436469672969, 0.5835486386473212, 0.6817246293079218, 0.7189080176546351, 0.7046335487088236, 0.6256304928832274, 0.7167663634820671, 0.6932145486759472, 0.7482269422487982]


## SVR

In [12]:
model=SVR(kernel='linear', C=1.25, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,7,model)

train_rmse: 0.004128782634767667
test_rmse: 0.00438183520690091
test_r2: 0.6562100947605988


## XGB

In [13]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'dart', 
              n_estimators = 150, 
              max_depth = 4, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.7,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.00444103519825924
[0.004736706282869011, 0.003959719593354398, 0.00516539278603524, 0.004412686053559277, 0.004385561722176456, 0.003929257794959692, 0.004602107848064806, 0.004274736968966771, 0.004975588078611608, 0.003968594853995149]


test_r2_ave: 0.6408239641609608
[0.542279491476767, 0.6847125512685877, 0.5182379313123588, 0.6696955175513484, 0.6314430236055604, 0.7364520214888184, 0.6060745769827518, 0.6931133214685656, 0.6037773307834808, 0.7224538756713693]


## RandomForest

In [14]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 7,
            min_samples_split = 6,
            max_leaf_nodes = 14,
            min_samples_leaf =1,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.004210428000843861
[0.00450619574494823, 0.0036921869255320053, 0.004845711920871969, 0.004237000178915273, 0.0038846800422618305, 0.003916908995091113, 0.004537172274781583, 0.0040239100765399214, 0.0045645390100714145, 0.0038959748394252766]


test_r2_ave: 0.6776289684220653
[0.5857451831114955, 0.7258771885721371, 0.5760241857269011, 0.6954733030797475, 0.7108223728018738, 0.7381059659833445, 0.6171126965494411, 0.7280708430104056, 0.6665395527904143, 0.7325183925948928]


In [8]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-05)
elastic_mod=ElasticNet(alpha=[0.006], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.25, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'dart', 
              n_estimators = 150, 
              max_depth = 4, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.7,
              reg_alpha =  0,
              reg_lambda = 1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 7,
            min_samples_split = 6,
            max_leaf_nodes = 14,
            min_samples_leaf =1,
            random_state = 42)  
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('RandomForest', random_mod),('XGB', xgb_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.004174705783125398
[0.0042938758429127396, 0.00373861833345558, 0.004853053530979205, 0.004259099184308161, 0.0038318950438594136, 0.004021453478086896, 0.00445210080119392, 0.004049164323298355, 0.004464168952233626, 0.0037836283409260854]


test_r2_ave: 0.6837144643705944
[0.6238626797777842, 0.718939328286418, 0.5747385034030223, 0.6922883670193725, 0.718627667055707, 0.7239392010707572, 0.6313362777173668, 0.7246468519893985, 0.6810433013583862, 0.7477224660277317]


In [9]:

stack_mod = StackingRegressor(regressors=[lasso_mod, lr, elastic_mod, random_mod], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.004178050778123379
[0.004327363712691984, 0.0037010497553692217, 0.004836017654954408, 0.004272674847008936, 0.003843871458823368, 0.003948444366646023, 0.004540864054920372, 0.004025299927752684, 0.004459622972275666, 0.0038252990307911226]


test_r2_ave: 0.6829512798777285
[0.6179728235218878, 0.7245595847746737, 0.5777188894210603, 0.6903236102641319, 0.7168660854316318, 0.7338719271828797, 0.6164893519923327, 0.7278829628980195, 0.6816925744945161, 0.742134988796153]


In [10]:
ave(skew_data[title],y,0.15,stack_mod,0.75,vote_mod,0.1,lasso_mod)

test_rmse_ave: 0.004162008155363495
[0.0042837736863567686, 0.0037233296536855857, 0.0048351382140715635, 0.0042489977141203115, 0.0038159286303579114, 0.004002692791382166, 0.004456968610771145, 0.0040371213596773845, 0.004444238171146485, 0.0037718927220656267]


test_r2_ave: 0.6855941021745885
[0.625630466440352, 0.7212333631768832, 0.5778724610296988, 0.6937462592485599, 0.7209675784921068, 0.7265089234709564, 0.6305296626140153, 0.7262823186587359, 0.6838849793105696, 0.749285009304007]


In [11]:
name='Input_C_057'
test_pred=[]
seed=[1,4,5,7,9,6,8]
stack_w=0.15
vote_w=0.75
model_w=0.1
model=lasso_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)


final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))
